# Bluebook for Bulldozers

### Introduction

### Loading our Data

In [5]:
import pandas as pd
df = pd.read_csv('./bulldozers.csv', index_col = 0)

In [6]:
df[:2]

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [19]:
object_df = df.select_dtypes('object')

In [20]:
object_df[:2]

,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,Low,11/16/2006 0:00,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,Low,3/26/2004 0:00,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


Now, many of these are categorical columns that we'll want our catboost regressor to take care of.  But the date column of sale date is something that we'll handle differently.

### Coercing the DateTime column

Let's start by selecting the column and coercing it to be of type `datetime`.

In [24]:
saledate_dt = pd.to_datetime(df.saledate)

saledate_dt.dtype

dtype('<M8[ns]')

Ok, now we can reassign it to our `df` dataframe, so that our original saledate column is replaced.

In [22]:
df_with_dt = df.assign(saledate = df.saledate)


In [23]:
df_with_dt.select_dtypes('object').columns

Index(['UsageBand', 'saledate', 'fiModelDesc', 'fiBaseModel',
       'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

So now the `saledate` is no longer of type object.

But our Catboost Regressor cannot handle features of type dtype, so let's use our `add_datepart` method to extract the relevant features from it.

In [25]:
import numpy as np
import re
def add_datepart(df, fldname, drop=True, time=False, errors="raise"):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [26]:
add_datepart(df_with_dt, 'saledate')

In [28]:
df_with_dt.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


We can see that we now have a lot more features related to the saledate.

### Missing Data?

Next we should identify our categorical data.

In [35]:
df_object = df_with_dt.select_dtypes(include = 'object')

In [36]:
df_object[:2]

,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,Low,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,Low,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


> Notice that there are a lot of values here, that are `na`, let's take care of them.  While Catboost *can* handle na columns, we'll want to create a Pool of our data, and this cannot.

In [90]:
any_na = df_with_dt.isna().sum()
na_cols = any_na[any_na != 0]
na_cols[:10]

UsageBand            1265
fiSecondaryDesc      3455
fiModelSeries        8624
fiModelDescriptor    7572
ProductSize          5370
Drive_System         7739
Enclosure               5
Forks                4939
Pad_Type             8309
Ride_Control         6651
dtype: int64

Let's check the lowest value across this dataset.

In [99]:
(df_with_dt[na_cols.index] == -999).any().any()
# False

False

Great, so we can use this to represent our na values.

In [100]:
df_no_na = df_with_dt.replace(np.nan, -999)
df_no_na[:2]

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,66000.0,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,57000.0,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200


In [102]:
df_no_na.isna().any().any()

False

For catboost, we need to identify the indices of this categorical data.  Let's use numpy to do so.

### Setting up our data

Now that everything is numeric or categorical, and we have identified our categorical features, it's time to split our data and create some datapools.  First, let's order our data by `saleElapsed`, so that we are evaluating our model by how well it predicts future data.

In [103]:
df_sorted = df_no_na.sort_values('saleElapsed')

In [104]:
df_sorted['saleYear'][:4]

7648    2004
8228    2004
6770    2004
4051    2004
Name: saleYear, dtype: int64

Assign `saleprice` to the variable `y` and every column *but* `saleprice` to the variable X.

In [105]:
X = df_sorted.drop('SalePrice', axis = 1)
y = df_sorted['SalePrice']

In [106]:
import numpy as np

cat_col_idcs = np.where(X.dtypes == np.object)[0]
cat_col_idcs

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

Ok, now let's split our data into training validation and test sets.  Make sure that `shuffle = False`, and make a 80-10-10 split of the data.

In [107]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, shuffle = False)
X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size = .5, shuffle = False)

In [108]:
X_train.shape
# (8000, 64)

(8000, 64)

In [109]:
X_train[:2]
# 	SalesID	MachineID	ModelID	datasource	auctioneerID	YearMade	MachineHoursCurrentMeter	UsageBand	fiModelDesc	fiBaseModel	...	saleDay	saleDayofweek	saleDayofyear	saleIs_month_end	saleIs_month_start	saleIs_quarter_end	saleIs_quarter_start	saleIs_year_end	saleIs_year_start	saleElapsed
# 7648	1165000	733687	7057	121	3.0	1995	4368.0	Medium	312	312	...	5	0	5	False	False	False	False	False	False	1073260800
# 8228	1166933	1035166	8861	121	3.0	2002	603.0	Low	803	803	...	9	4	9	False	False	False	False	False	False	1073606400
# 2 rows × 64 columns

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
7648,1165000,733687,7057,121,3.0,1995,4368.0,Medium,312,312,...,5,0,5,False,False,False,False,False,False,1073260800
8228,1166933,1035166,8861,121,3.0,2002,603.0,Low,803,803,...,9,4,9,False,False,False,False,False,False,1073606400


In [110]:
X_validate[:2]

# 	SalesID	MachineID	ModelID	datasource	auctioneerID	YearMade	MachineHoursCurrentMeter	UsageBand	fiModelDesc	fiBaseModel	...	saleDay	saleDayofweek	saleDayofyear	saleIs_month_end	saleIs_month_start	saleIs_quarter_end	saleIs_quarter_start	saleIs_year_end	saleIs_year_start	saleElapsed
# 8607	1168308	1067786	4875	121	3.0	1000	4075.0	Low	520C	520	...	28	3	28	False	False	False	False	False	False	1264636800
# 7244	1163639	1053311	3357	121	3.0	1000	0.0	NaN	12G	12	...	28	3	28	False	False	False	False	False	False	1264636800
# 2 rows × 64 columns

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
8607,1168308,1067786,4875,121,3.0,1000,4075.0,Low,520C,520,...,28,3,28,False,False,False,False,False,False,1264636800
7244,1163639,1053311,3357,121,3.0,1000,0.0,-999,12G,12,...,28,3,28,False,False,False,False,False,False,1264636800


In [111]:
X_train.shape, X_validate.shape, X_test.shape

((8000, 64), (1000, 64), (1000, 64))

Now let's create data pools for our train, validation and test data.

In [115]:
cat_col_idcs = np.where(X_train.dtypes == 'object')[0]
cat_col_idcs

array([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
       41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [117]:
from catboost import Pool

train_pool = Pool(X_train, y_train, cat_features = cat_col_idcs)
validate_pool = Pool(X_validate, y_validate, cat_features = cat_col_idcs)
test_pool = Pool(X_test, y_test, cat_features = cat_col_idcs)

Now let's train our regressor.

In [119]:
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(iterations = 200, depth = 6,  
                        logging_level='Silent', 
                        random_seed = 42).fit(train_pool)

cbr.score(validate_pool)

0.8157309442547013

In [ ]:
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'min_child_samples': 7,
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

{'iterations': 500,
 'learning_rate': 0.1,
 'loss_function': 'RMSE',
 'random_seed': 42,
 'logging_level': 'Silent',
 'eval_metric': 'RMSE',
 'min_child_samples': 7}

In [31]:
from catboost import CatBoostRegressor

CatBoostRegressor()

### Resources

[Cross entropy vs log loss](https://jamesmccaffrey.wordpress.com/2016/09/25/log-loss-and-cross-entropy-are-almost-the-same/#:~:text=Log%20loss%20is%20usually%20used,three%20or%20more%20possible%20outcomes.&text=In%20words%2C%20cross%20entropy%20is,probabilities%20times%20the%20actual%20probabilities.)